In [1]:
!pwd

/root/nikitam/kaggle/APTOS_BLINDNESS/CATALYST_PIPELINE


In [2]:
!cp ../CATALYST_PIPELINE /home

cp: -r not specified; omitting directory '../CATALYST_PIPELINE'


In [1]:
# !cp logs/efficientnet-4/finetuning_combined_256/checkpoints/last.pth effnet4_256_pretrain_and_finetuning.last.pth

In [2]:
from helpers import *

from albumentations import Compose, Flip

In [4]:
cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)

In [5]:
path_to_data = "data/test_images"
path_to_csv = "data/sample_submission.csv"
sample = pd.read_csv(path_to_csv)

coefficients=[0.5, 1.5, 2.5, 3.5]

## Predict

In [6]:
path_to_checkpoint = "effnet4_256_pretrain_and_finetuning.last.pth"

image_size = 256
k=4
batch_size = 8
num_workers = 4
crop_from_gray = True
circle_crop = True
ben_preprocess = 10
normalize = True
device = torch.device("cuda:0")

checkpoint = torch.load(path_to_checkpoint)
model = efficientnet_pretrained(k, num_classes=1, pretrained=False)
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
model.eval();

test_transforms = Compose(
    [pre_transforms(image_size, crop_from_gray, circle_crop, ben_preprocess),
     Compose([Flip()]),
     post_transforms(normalize)
    ])
test_dataset = RetinopathyDatasetTest(csv_file=path_to_csv,
                                      path_to_data=path_to_data,
                                      transform=test_transforms)
test_dl = torch.utils.data.DataLoader(test_dataset,
                                      batch_size=batch_size,
                                      shuffle=False,
                                      num_workers=num_workers)

preds = predict(test_dl, model, batch_size, len(test_dataset), device)

sample.diagnosis = round_preds(preds).astype(int)
sample.to_csv("submission.csv", index=False)

100%|██████████| 241/241 [00:37<00:00,  7.12it/s]


In [7]:
sample.diagnosis.value_counts()

2    1051
0     356
3     308
1     166
4      47
Name: diagnosis, dtype: int64